In [5]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')

# url for repliseq exps
exp_types = ['NAD-seq']
set_url = '/search/?'+'&'.join(['experiments_in_set.experiment_type='+i for i in exp_types])+ \
          '&type=ExperimentSetReplicate&limit=all' + \
          '&status=pre-release&status=released&status=released%20to%20project'
run_sets = ff_utils.search_metadata(set_url ,key=my_auth)


all_sets = len(run_sets)
print(str(all_sets)+' total number of sets')

run_sets = [i for i in run_sets if "RepliSeq_Pipeline_v13.1_step1"  not in i.get('completed_processes', [])]
run_sets = [i for i in run_sets if "RepliSeq_Pipeline_v14_step1"  not in i.get('completed_processes', [])]
run_sets = [i for i in run_sets if "RepliSeq_Pipeline_v16_step1"  not in i.get('completed_processes', [])]

print(str(all_sets-len(run_sets))+ ' sets completed')



12 total number of sets
0 sets completed


In [6]:

add_pc = False
add_tag = False
add_wfr = True
counter = 0
for a_set in run_sets: 
    print(a_set['description'])
    counter += 1

    
    print()
    # run in single mode
    paired = ""
    try:
        fastqpairs, organism, enzyme, bwa_ref, chrsize_ref, enz_ref, f_size, lab = find_pairs(a_set, my_env, lookfor = 'single')
        paired = 'No'
    # run in paired mode
    except:
        fastqpairs, organism, enzyme, bwa_ref, chrsize_ref, enz_ref, f_size, lab = find_pairs(a_set, my_env)
        paired = 'Yes'
    if not bwa_ref or not chrsize_ref:
        print(counter, a_set['accession'], organism, enzyme, 'skipping set with not chrsize/bwa index')
        continue  
    print(counter, a_set['accession'], enzyme, organism)      
    # cycle through the experiments
    for exp in fastqpairs.keys():
        if not fastqpairs.get(exp):
            print(exp, 'does not have any fastq files')
            continue
        # Check Part 1 and See if all are okay
        exp_bams = []
        part1 = 'done'
        # part2 = 'done'
        for pair in fastqpairs[exp]:
            #############
            if paired == 'No':
                pair_resp = ff_utils.get_metadata(pair, key=my_auth)
                report = get_wfr_out(pair, 'repliseq-parta', my_auth, ['v13.1', 'v14', 'v16'])
            if paired == 'Yes':
                pair_resp = ff_utils.get_metadata(pair[0], key=my_auth)
                report = get_wfr_out(pair[0], 'repliseq-parta', my_auth, ['v13.1', 'v14', 'v16'])
                
            attributions = get_attribution(pair_resp)
            # if run is not successful
            if report['status'].startswith('no'):
                part1 = 'not done'
                if add_wfr:
                    if paired == 'Yes':
                        inp_f = {'fastq':pair[0], 'fastq2': pair[1], 'chromsizes':chrsize_ref, 'bwaIndex':bwa_ref}
                        name_tag = pair[0].split('/')[2] + '_' + pair[1].split('/')[2]
                    if paired == 'No':
                        inp_f = {'fastq':pair, 'chromsizes':chrsize_ref, 'bwaIndex':bwa_ref}
                        name_tag = pair.split('/')[2]
                    run_missing_wfr(step_settings('repliseq-parta', organism, attributions), inp_f, name_tag,my_auth, my_env)
            elif report['status'] == 'running':
                part1 = 'still running'
                print('part1 still running')
            # if successful
            else:
                assert report['status'] == 'complete'
                if add_pc:
                    # TODO check if the files already in processed files field.
                    # don't do it if they are already carried there.
                    print('Adding files to preliminary tab')
                    add_preliminary_processed_files(exp,
                                                    [report['bam'], report['bg']],
                                                    my_auth,
                                                    run_type='repliseq')
                if add_tag:
                    # get version
                    res_file_resp = ff_utils.get_metadata(report['bam'], my_auth)
                    producing_wfr = res_file_resp['workflow_run_outputs'][0]['display_title']
                    wfr_type, time_info = producing_wfr.split(' run ')
                    wfr_type_base, wfr_version = wfr_type.strip().split(' ')
                    assert wfr_version in ['v13.1', 'v14', 'v16']
                    tag = 'RepliSeq_Pipeline_{}_step1'.format(wfr_version)
                    ff_utils.patch_metadata({"completed_processes":[tag]}, obj_id=a_set['accession'] , key=my_auth)
                     
        # stop progress to part2 
        if part1 is not 'done':
            print(exp, 'has missing Part1 runs')
            part2 = 'not ready'
            part3 = 'not ready'
            continue
        print(exp, 'part1 complete')
        print(report)
        


DNAseq of nucleolar-associated DNA from crosslinked F121-9 cells

expected single files, found paired end
1 4DNESXE9K9DB None mouse
4DNEX5KX1UNW part1 complete
{'status': 'complete', 'bg': '/files-processed/4DNFIDXS12ES/', 'bam': '/files-processed/4DNFIJR7CHTT/'}
4DNEXMU2C5YD part1 complete
{'status': 'complete', 'bg': '/files-processed/4DNFI8KF8VRE/', 'bam': '/files-processed/4DNFIAEWXF7C/'}
WGS seq of total genomic DNA from crosslinked F121-9 cells

expected single files, found paired end
2 4DNESUJZ5FL2 None mouse
4DNEX14H1J99 part1 complete
{'status': 'complete', 'bg': '/files-processed/4DNFIAX32I9H/', 'bam': '/files-processed/4DNFIRQIWK72/'}
4DNEXRAWM3QN part1 complete
{'status': 'complete', 'bg': '/files-processed/4DNFICZUB8LQ/', 'bam': '/files-processed/4DNFI18EHRID/'}
DNAseq of nucleolar-associated DNA from non-crosslinked MEF cells (form UMASS core)

expected single files, found paired end
3 4DNES15QV1OO None mouse


4DNEXCET6LVA has missing Part1 runs
WGS seq of total genomic DNA from non-crosslinked MEF cells (from UMASS core)

expected single files, found paired end
4 4DNES8D85R8H None mouse


4DNEXVDUDPJN has missing Part1 runs
DNAseq of nucleolar-associated DNA from crosslinked H1 cells

expected single files, found paired end
5 4DNESV4F9PFH None human


4DNEXQ15J5UJ has missing Part1 runs
WGS seq of total genomic DNA from crosslinked H1 cells

expected single files, found paired end
6 4DNESV9Q2PMB None human


4DNEXUEPMOTS has missing Part1 runs
DNAseq of nucleolar-associated DNA from crosslinked HFFc6 cells

expected single files, found paired end
7 4DNESVA79ZRN None human


...

4DNEXO4ZBUGU has missing Part1 runs
DNAseq of nucleolar-associated DNA from non-crosslinked MEF cells

expected single files, found paired end
11 4DNESNEBF9W4 None mouse


4DNEXSHU18RN has missing Part1 runs


4DNEXOHSK51Y has missing Part1 runs
WGS seq of total genomic DNA from non-crosslinked MEF cells

expected single files, found paired end
12 4DNES7SVJXAI None mouse


4DNEX993GF46 has missing Part1 runs


4DNEX4OLAN3C has missing Part1 runs


In [11]:
# Move files from opc to pc
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

action = True

# get key from keypairs.json
my_key = get_key('koray_data')
# which sets to move data
set_url = '/search/?completed_processes=RepliSeq_Pipeline_V16_step1%27&completed_processes=RepliSeq_Pipeline_v16_step1&experiments_in_set.experiment_type=Repli-seq&limit=all&type=ExperimentSetReplicate'
run_sets = ff_utils.search_metadata(set_url, my_key)
print(len(run_sets))
# move other processed files to processed files field
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        print('There are files in processed_files field, expected empty')
        return False
    # are there files in opc
    if not opc:
        print('there are no other processed files, skipping')
        return False
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            if action: 
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                # update status of pc to status of set or exp
                release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

        
        


set_w_apf = 0
exp_w_apf = 0
counter = 0
#move_title = 'HiC Processing Pipeline - Preliminary Files'
move_title = "Repli-Seq Pipeline - Preliminary Files"

print(len(run_sets), 'experiment sets in scope')
for a_set in run_sets:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_key, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_key)
    if res:
        set_w_apf += 1
        print(set_resp['accession'], 'moved to pc')
  
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_key, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_key)
        if res_e:
            exp_w_apf += 1
            print(exp_resp['accession'], 'moved to pc')
    print()

print(set_w_apf)
print(exp_w_apf)

4
4 experiment sets in scope
1 4DNESHLNICHQ
there are no other processed files, skipping
There are files in processed_files field, expected empty
4DNEXNZUFKAN files will move
4DNEXNZUFKAN moved to pc

2 4DNESC15J6HU
there are no other processed files, skipping
4DNEX56YP5JV files will move
4DNEX56YP5JV moved to pc
4DNEXC78ZQ5W files will move
4DNEXC78ZQ5W moved to pc

3 4DNES4WLR8MV
there are no other processed files, skipping
4DNEXJHA5PYA files will move
4DNEXJHA5PYA moved to pc
4DNEX17H5YRC files will move
4DNEX17H5YRC moved to pc

4 4DNESGWKYYWO
there are no other processed files, skipping
4DNEX3RCXAVO files will move
4DNEX3RCXAVO moved to pc
4DNEX5WSFYYF files will move
4DNEX5WSFYYF moved to pc

0
7


In [ ]:
# Repliseq pipeline part 1 also produces a qc object on the files.
# Change their status to the ones of the files (if the files are in processed files field)

# set_url = '/search/?experiments_in_set.experiment_type=Repli-seq&type=ExperimentSetReplicate&limit=all'

# set_url = '/search/?type=ExperimentSetReplicate&experiments_in_set.experiment_type=Repli-seq&experiments_in_set.biosample.biosource.individual.organism.name=mouse'


# run_sets = ff_utils.search_metadata(set_url , key=my_key)

def release_qc(file_id, con_key):
    file_resp = ff_utils.get_metadata(file_id, key=con_key, add_on = 'frame=raw')
    file_status = file_resp['status']
    # skip if file is not released/archived
    if file_status not in ['released', 'released to project', 'archived']:
        return
    qc = file_resp.get('quality_metric')
    if qc:
        qc_resp = ff_utils.get_metadata(qc, key=con_key)
        qc_status = qc_resp['status']
    else:
        return
    
    if qc_status != file_status:
        ff_utils.patch_metadata({'status': file_status}, obj_id=qc, key=con_key)
        print(qc, 'qc object updated with status', file_status)
        return True
    
print(len(run_sets), 'experiment sets in scope')
for a_set in run_sets:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_key, add_on='frame=raw')
    if set_resp.get('processed_files'):
        for a_pf in set_resp['processed_files']:
            release_qc(a_pf, my_key)
            
    exps = set_resp['experiments_in_set']
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_key, add_on='frame=raw')
        if exp_resp.get('processed_files'):
            for a_pf_e in exp_resp['processed_files']:
                release_qc(a_pf_e, my_key)